In [11]:
from xml.dom import minidom
import numpy as np
import geopy.distance
import pandas as pd

# Query feita (foram duas):
"""
<query type="way">
    <around lat="-23.5585875" lon="-46.6686528" radius="800" />
    <has-kv k="highway"/>
  	<has-kv k="oneway"/>
  	<has-kv k="maxspeed"/>
</query>
<union>
  <item/>
  <recurse type="down"/>
</union>
<print>
</print>

<query type="way">
    <around lat="-23.5630646" lon="-46.6865378" radius="800" />
    <has-kv k="highway"/>
  	<has-kv k="oneway"/>
  	<has-kv k="maxspeed"/>
</query>
<union>
  <item/>
  <recurse type="down"/>
</union>
<print>
</print>
"""

EVENTS_OVERPASS_MAP_PATH = "events_edges_overpass_query"
MAP_COMPLETE_OVERPASS_MAP_PATH = "map_complete_edges_overpass_query"
events_xml = minidom.parse(EVENTS_OVERPASS_MAP_PATH)
map_xml = minidom.parse(MAP_COMPLETE_OVERPASS_MAP_PATH)

# Arestas que sofrerão eventos de fechamento

![Mapa com Arestas Utilizado](events_edges_map.png)

In [12]:
def chunk_nodes(nodes):
    """
    Usage:
    > chunk_nodes([1,2,3,4,5])
    [(1, 2), (2, 3), (3, 4), (4, 5)]
    """
    list1 = list(nodes)[:-1]
    list2 = list(nodes)[1:]
    
    return list(zip(list1, list2))

In [13]:
events_ways_tag = events_xml.getElementsByTagName('way')
map_ways_tag = map_xml.getElementsByTagName('way')

events_nodes_tags = events_xml.getElementsByTagName('node')
map_nodes_tags = events_xml.getElementsByTagName('node')

events_nodes = {}
events_edges = []
map_nodes = {}
map_edges = {}

In [14]:
for n in map_nodes_tags:
    map_nodes[int(n.getAttribute('id'))] = (float(n.getAttribute('lat')), float(n.getAttribute('lon')))
    
for n in events_nodes_tags:
    events_nodes[int(n.getAttribute('id'))] = (float(n.getAttribute('lat')), float(n.getAttribute('lon')))

In [15]:
for w in events_ways_tag:
    events_edges.append(int(w.getAttribute('id')))

In [16]:
events_edges_related = []

In [17]:
highest = 0
counter = 1
for w in map_ways_tag:
    tags = w.getElementsByTagName('tag')
    width = 2
    maxspeed = 0
    oneway = 1
    
    for t in tags:
        key = t.getAttribute('k')
        value = t.getAttribute('v')
        
        if (key == 'width'):
            width = float(value)
        elif (key == 'maxspeed'):
            maxspeed = int(value)/3.6
        elif (key == 'oneway'):
            oneway = 1 if (value == 'yes') else 0
    
    chunks = chunk_nodes(w.getElementsByTagName('nd'))
    for chunk in chunks:
        n1 = chunk[0].getAttribute('ref')
        n2 = chunk[1].getAttribute('ref')
        if (int(w.getAttribute('id')) in events_edges):
            events_edges_related.append([w.getAttribute('id'), n1, n2, counter])
        counter += 1

In [18]:
df = pd.DataFrame(columns=["type", "from", "to", "start_tick", "duration", "slowdown"])
counter = 0

for e in events_edges_related:
    df.loc[counter] = ["close_street", e[1], e[2], 100, 1700, 0]
    counter += 1

In [19]:
df.describe()

,type,from,to,start_tick,duration,slowdown
count,984,984,984,984,984,984
unique,1,909,907,1,1,1
top,close_street,206274836,1832159347,1,1100,0
freq,984,3,3,984,984,984


In [20]:
df.to_csv("events.csv", header=False, sep=";", index=False)